# Test Loss Functions on Multiple Models

In [2]:
import sys
sys.path.append('src')
import pandas as pd
import numpy as np
import xgboost as xg
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import yaml
# Local modules
from fmda_models import LM, XGB, MLP
import reproducibility

In [5]:
models = {
    'lm': '...',
    'xgb': '...',
    'mlp': '...'
}
dat = {
    'mse': {
        'func': mean_squared_error,
        'models': models
    },
    'ros': {
        'func': '...',
        'models': models
    }
}